Grid Board Q-Learning
---
<img src="board.png" alt="drawing" width="600"/>

---
When the agent moving up, it has **0.8** probability moving up and **0.1** probability of going left or right. (non-deterministic)
<img style="float:left" src="Q-steps.png" alt="drawing" width="600"/>


In [1]:
import numpy as np

In [2]:
BOARD_ROWS = 3
BOARD_COLS = 4
WIN_STATE = (0, 3)
LOSE_STATE = (1, 3)
START = (2, 0)
DETERMINISTIC = False

In [3]:
class State:
    def __init__(self, state=START):
        self.board = np.zeros([BOARD_ROWS, BOARD_COLS])
        self.board[1, 1] = -1
        self.state = state
        self.isEnd = False
        self.determine = DETERMINISTIC
        
    def giveReward(self):
        if self.state == WIN_STATE:
            return 1
        elif self.state == LOSE_STATE:
            return -1
        else:
            return 0
    
    def isEndFunc(self):
        if (self.state == WIN_STATE) or (self.state == LOSE_STATE):
            self.isEnd = True

    def _chooseActionProb(self, action):
        if action == "up":
            return np.random.choice(["up", "left", "right"], p=[0.8, 0.1, 0.1])
        if action == "down":
            return np.random.choice(["down", "left", "right"], p=[0.8, 0.1, 0.1])
        if action == "left":
            return np.random.choice(["left", "up", "down"], p=[0.8, 0.1, 0.1])
        if action == "right":
            return np.random.choice(["right", "up", "down"], p=[0.8, 0.1, 0.1])
        
    def nxtPosition(self, action):
        """
        action: up, down, left, right
        -------------
        0 | 1 | 2| 3|
        1 |
        2 |
        return next position on board
        """
        if self.determine:
            if action == "up":
                nxtState = (self.state[0]-1, self.state[1])
            elif action == "down":
                nxtState = (self.state[0]+1, self.state[1])
            elif action == "left":
                nxtState = (self.state[0], self.state[1]-1)
            else:
                nxtState = (self.state[0], self.state[1]+1)
            self.determine = False
        else:
            # non-deterministic
            action = self._chooseActionProb(action)
            self.determine = True
            nxtState = self.nxtPosition(action)
                        
        # if next state is legal
        if (nxtState[0] >= 0) and (nxtState[0] <= 2):
            if (nxtState[1] >= 0) and (nxtState[1] <= 3):
                if nxtState != (1, 1):
                    return nxtState
        return self.state
    
    def showBoard(self):
        self.board[self.state] = 1
        for i in range(0, BOARD_ROWS):
            print('-----------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = '*'
                if self.board[i, j] == -1:
                    token = 'z'
                if self.board[i, j] == 0:
                    token = '0'
                out += token + ' | '
            print(out)
        print('-----------------')    

In [4]:
class Agent:
    
    def __init__(self):
        self.states = []  # record position and action taken at the position
        self.actions = ["up", "down", "left", "right"]
        self.State = State()
        self.isEnd = self.State.isEnd
        self.lr = 0.2
        self.exp_rate = 0.3
        self.decay_gamma = 0.9

        # initial Q values
        self.Q_values = {}
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                self.Q_values[(i, j)] = {}
                for a in self.actions:
                    self.Q_values[(i, j)][a] = 0  # Q value is a dict of dict  
    
    def chooseAction(self):
        # choose action with most expected value
        mx_nxt_reward = 0
        action = ""
        
        if np.random.uniform(0, 1) <= self.exp_rate:
            action = np.random.choice(self.actions)
        else:
            # greedy action
            for a in self.actions:
                current_position = self.State.state
                nxt_reward = self.Q_values[current_position][a]
                if nxt_reward >= mx_nxt_reward:
                    action = a
                    mx_nxt_reward = nxt_reward
            # print("current pos: {}, greedy aciton: {}".format(self.State.state, action))
        return action
    
    def takeAction(self, action):
        position = self.State.nxtPosition(action)
        # update State
        return State(state=position)     
    
    def reset(self):
        self.states = []
        self.State = State()
        self.isEnd = self.State.isEnd
    
    def play(self, rounds=10):
        i = 0
        while i < rounds:
            # to the end of game back propagate reward
            if self.State.isEnd:
                # back propagate
                reward = self.State.giveReward()
                for a in self.actions:
                    self.Q_values[self.State.state][a] = reward
                print("Game End Reward", reward)
                for s in reversed(self.states):
                    current_q_value = self.Q_values[s[0]][s[1]]
                    reward = current_q_value + self.lr*(self.decay_gamma*reward - current_q_value)
                    self.Q_values[s[0]][s[1]] = round(reward, 3)
                self.reset()
                i += 1
            else:
                action = self.chooseAction()
                # append trace
                self.states.append([(self.State.state), action])
                print("current position {} action {}".format(self.State.state, action))
                # by taking the action, it reaches the next state
                self.State = self.takeAction(action)
                # mark is end
                self.State.isEndFunc()
                print("nxt state", self.State.state)
                print("---------------------")
                self.isEnd = self.State.isEnd

In [5]:
ag = Agent()

In [6]:
ag.Q_values

{(0, 0): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (0, 1): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (0, 2): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (0, 3): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (1, 0): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (1, 1): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (1, 2): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (1, 3): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (2, 0): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (2, 1): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (2, 2): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (2, 3): {'up': 0, 'down': 0, 'left': 0, 'right': 0}}

In [7]:
ag.play(50)

current position (2, 0) action right
nxt state (2, 0)
---------------------
current position (2, 0) action right
nxt state (2, 1)
---------------------
current position (2, 1) action right
nxt state (2, 1)
---------------------
current position (2, 1) action down
nxt state (2, 0)
---------------------
current position (2, 0) action up
nxt state (1, 0)
---------------------
current position (1, 0) action right
nxt state (1, 0)
---------------------
current position (1, 0) action right
nxt state (1, 0)
---------------------
current position (1, 0) action down
nxt state (2, 0)
---------------------
current position (2, 0) action right
nxt state (2, 1)
---------------------
current position (2, 1) action up
nxt state (2, 2)
---------------------
current position (2, 2) action right
nxt state (2, 3)
---------------------
current position (2, 3) action up
nxt state (1, 3)
---------------------
Game End Reward -1
current position (2, 0) action left
nxt state (2, 0)
---------------------
curre

nxt state (0, 2)
---------------------
current position (0, 2) action right
nxt state (0, 3)
---------------------
Game End Reward 1
current position (2, 0) action right
nxt state (2, 1)
---------------------
current position (2, 1) action down
nxt state (2, 1)
---------------------
current position (2, 1) action left
nxt state (2, 1)
---------------------
current position (2, 1) action left
nxt state (2, 0)
---------------------
current position (2, 0) action right
nxt state (2, 1)
---------------------
current position (2, 1) action down
nxt state (2, 1)
---------------------
current position (2, 1) action left
nxt state (2, 0)
---------------------
current position (2, 0) action up
nxt state (2, 0)
---------------------
current position (2, 0) action up
nxt state (1, 0)
---------------------
current position (1, 0) action up
nxt state (1, 0)
---------------------
current position (1, 0) action down
nxt state (2, 0)
---------------------
current position (2, 0) action up
nxt state (1

In [8]:
ag.Q_values

{(0, 0): {'up': 0.32, 'down': 0.216, 'left': 0.191, 'right': 0.622},
 (0, 1): {'up': 0.464, 'down': 0.209, 'left': 0.194, 'right': 0.755},
 (0, 2): {'up': 0.339, 'down': 0.042, 'left': 0.262, 'right': 0.871},
 (0, 3): {'up': 1, 'down': 1, 'left': 1, 'right': 1},
 (1, 0): {'up': 0.379, 'down': 0.107, 'left': 0.038, 'right': 0.187},
 (1, 1): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (1, 2): {'up': 0.167, 'down': 0, 'left': 0.066, 'right': -0.18},
 (1, 3): {'up': -1, 'down': -1, 'left': -1, 'right': -1},
 (2, 0): {'up': 0.209, 'down': 0.114, 'left': 0.056, 'right': 0.05},
 (2, 1): {'up': 0.036, 'down': 0.039, 'left': 0.109, 'right': 0.004},
 (2, 2): {'up': 0, 'down': 0.007, 'left': 0.038, 'right': -0.032},
 (2, 3): {'up': -0.18, 'down': 0, 'left': 0, 'right': 0}}